# Capstone

Michael Schillawski, 10 April 2018

Data Science Immersive, Generaly Assembly

## Imports

In [1]:
import os
import json
import re
import string
import multiprocessing

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from pandas.io.json import json_normalize
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from joblib import Parallel, delayed


%matplotlib inline

os.getcwd()

'/Users/mjschillawski/Google Drive/Data/generalassembly/projects/GitHub Portfolio/capstone_project'

## Load Data

In [2]:
path = '/Users/mjschillawski/Desktop/Miscellaneous Data/Yummly28K/'
file = 'data_records_27638.txt'

data = pd.read_table(path+file,header=None,names=['recipe'],index_col=1)

In [3]:
data.head()

,recipe
1,Mushroom-risotto-352492
2,Filipino-bbq-pork-skewers-352163
3,Mushroom-and-Roasted-Garlic-Risotto-525026
4,Gratin-Dauphinois-_scalloped-Potatoes-With-Che...
5,Delicious-Grilled-Hamburgers-Allrecipes


In [4]:
data.tail()

,recipe
27634,Saltimbocca-alla-Romana-824004
27635,Carrotini-MyRecipes-241041
27636,Brioche-531830
27637,Mexican-Hot-Chocolate-Doughnuts-501510
27638,Thai-Curry-Burrito-Bowl-with-Sweet-and-Spicy-T...


In [5]:
path = '/Users/mjschillawski/Desktop/Miscellaneous Data/Yummly28K/metadata27638/'

recipes = []

for i in data.index:
    num = str(i)
    while len(num) < 5:
        num = '0' + num
        
    # https://stackoverflow.com/questions/28373282/how-to-read-a-json-dictionary-type-file-with-pandas
    with open(path+'meta'+num+'.json') as json_data:
        recipe = json.load(json_data)
        recipes.append(recipe)

recipes = json_normalize(recipes)   
recipes

,attributes.course,attributes.cuisine,attributes.holiday,attribution.html,attribution.logo,attribution.text,attribution.url,cookTime,cookTimeInSeconds,flavors.Bitter,...,nutritionEstimates,prepTime,prepTimeInSeconds,rating,source.sourceDisplayName,source.sourceRecipeUrl,source.sourceSiteUrl,totalTime,totalTimeInSeconds,yield
0,[Side Dishes],[Italian],NaN,<a href='http://www.yummly.com/recipe/Mushroom...,http://static.yummly.com/api-logo.png,Mushroom Risotto recipes: information powered ...,http://www.yummly.com/recipe/Mushroom-risotto-...,NaN,NaN,NaN,...,"[{'attribute': 'FAT_KCAL', 'unit': {'name': 'c...",NaN,NaN,5,Skinnytaste,http://www.skinnytaste.com/2009/10/risotto-is-...,http://www.skinnytaste.com,30 minutes,1800.0,servings: 6
1,[Main Dishes],[Barbecue],NaN,<a href='http://www.yummly.com/recipe/Filipino...,http://static.yummly.com/api-logo.png,Filipino BBQ Pork Skewers recipes: information...,http://www.yummly.com/recipe/Filipino-bbq-pork...,NaN,NaN,0.8333,...,"[{'attribute': 'FAT_KCAL', 'unit': {'name': 'c...",NaN,NaN,5,Skinnytaste,http://www.skinnytaste.com/2008/08/filipino-bb...,http://www.skinnytaste.com,40 min,2400.0,None
2,[Main Dishes],[Italian],NaN,<a href='http://www.yummly.com/recipe/Mushroom...,http://static.yummly.com/api-logo.png,Mushroom and Roasted Garlic Risotto recipes: i...,http://www.yummly.com/recipe/Mushroom-and-Roas...,NaN,NaN,1.0000,...,"[{'attribute': 'FAT_KCAL', 'unit': {'name': 'c...",NaN,NaN,3,MyRecipes,http://www.myrecipes.com/recipe/mushroom-roast...,http://www.myrecipes.com,1 Hr 25 Min,5100.0,Serves 6 (serving size: about 1 cup)
3,[Side Dishes],"[French, American]",NaN,<a href='http://www.yummly.com/recipe/Gratin-D...,http://static.yummly.com/api-logo.png,Gratin Dauphinois (Scalloped Potatoes with Che...,http://www.yummly.com/recipe/Gratin-Dauphinois...,NaN,NaN,0.6667,...,"[{'attribute': 'FAT_KCAL', 'unit': {'name': 'c...",NaN,NaN,4,MyRecipes,http://www.myrecipes.com/recipe/gratin-dauphin...,http://www.myrecipes.com,55 min,3300.0,7 servings (serving size: 1 cup)
4,[Main Dishes],[Barbecue],NaN,<a href='http://www.yummly.com/recipe/Deliciou...,http://static.yummly.com/api-logo.png,Delicious Grilled Hamburgers recipes: informat...,http://www.yummly.com/recipe/Delicious-Grilled...,10 Min,600.0,0.1667,...,"[{'attribute': 'FAT_KCAL', 'unit': {'name': 'c...",5 Min,300.0,4,AllRecipes,http://allrecipes.com/Recipe/delicious-grilled...,http://www.allrecipes.com,15 Min,900.0,3 servings
5,[Main Dishes],[American],"[Winter, Christmas, Thanksgiving]",<a href='http://www.yummly.com/recipe/Hickory_...,http://static.yummly.com/api-logo.png,Hickory-Smoked Bourbon Turkey recipes: informa...,http://www.yummly.com/recipe/Hickory_smoked-Bo...,NaN,NaN,0.8333,...,"[{'attribute': 'FAT_KCAL', 'unit': {'name': 'c...",NaN,NaN,4,MyRecipes,http://www.myrecipes.com/recipe/hickory-smoked...,http://www.myrecipes.com,None,NaN,Makes 12 to 14 servings
6,[Main Dishes],[American],NaN,<a href='http://www.yummly.com/recipe/Marinate...,http://static.yummly.com/api-logo.png,Marinated Beef Tenderloin recipes: information...,http://www.yummly.com/recipe/Marinated-Beef-Te...,NaN,NaN,0.6667,...,"[{'attribute': 'FAT_KCAL', 'unit': {'name': 'c...",25 Min,1500.0,3,Martha Stewart,http://www.marthastewart.com/962284/marinated-...,http://www.marthastewart.com,3 Hr 30 Min,12600.0,servings: 8
7,[Desserts],[Italian],[Summer],<a href='http://www.yummly.com/recipe/Cherry-T...,http://static.yummly.com/api-logo.png,Cherry Tortoni recipes: information powered by...,http://www.yummly.com/recipe/Cherry-Tortoni-My...,NaN,NaN,0.3333,...,"[{'attribute': 'FAT_KCAL', 'unit': {'name': 'c...",NaN,NaN,3,MyRecipes,http://www.myrecipes.com/recipe/cherry-tortoni...,http://www.myrecipes.com,None,NaN,10 servings (serving size: 1 [3-inch] tortoni ...
8,[Main Dishes],[Italian],NaN,<a href='http://www.yummly.com/recipe/Crock-Po...,http://static.yummly.com/api-logo.png,Crock-Pot Italian Turkey Meatballs and Homemad...,http://www.yummly.com/recipe/Crock-Pot-Italian...,NaN

### EDA

In [ ]:
# distribution of courses

In [ ]:
# distribution of cuisines

### Extract columns we care about

In [8]:
short_recipes = recipes[['attributes.course','attributes.cuisine','name','ingredientLines']]

In [9]:
short_recipes['ingredientLines'][17464]

['9 ounces farro perlato (semi-pearled emmer), about 1 1/3 cups uncooked (see substitutes below)',
 '1 teaspoon chia seed or ground flaxseeds',
 '3 medium zucchini, trimmed and halved lengthwise',
 '1/2 red bell pepper, seeded',
 '12-16 ounces cherry tomatoes, halved',
 '2 green onions, green parts only, thinly sliced',
 '2 tablespoons white balsamic or white wine vinegar',
 '1 tablespoon lemon juice',
 '2 teaspoons spicy or Dijon mustard, or to taste',
 '1 large clove garlic, minced or pressed',
 '1/4 teaspoon salt (optional)',
 '1/8 teaspoon freshly ground black pepper',
 '1/4 cup chopped fresh basil',
 'Salt and black pepper, to taste',
 'Additional lemon juice, to taste']

## Natural Language Processing

strip numbers

strip common measurements and their abbreviations

strip punctuation

strip preparation methods

strip 'to taste'

inventory my spice rack

strip spices / pantry items

In [22]:
# single-threaded
def process_ingredients(recipes,nondescript=0,drop_words=None):
    # create the patterns

    # import punctuation characters
    # to remove all punctuation
    punct = string.punctuation
    punct_pattern = r"[{}]".format(punct)

    # to remove all numbers
    number_pattern = r"\d+\s"
    
    # embedded numbers
    embed_num_pattern = r".\d+."
    
    # removed prep methods
    prep_pattern = r"[a-z]+ed"
    
    # strip pluralization
    plural_pattern = r"s\s"
    recipes_ingredients = []

    for recipe in recipes:
        ingredients = []
    
        for item in recipe:

            # strip punctuation
            text = re.sub(punct_pattern," ",item)
            # strip standalone numbers
            text = re.sub(number_pattern,"",text)
            # strip embedded numbers
            text = re.sub(embed_num_pattern,"",text)
            # strip preparation methods
            text = re.sub(prep_pattern,"",text)
            # strip pluralization
            text = re.sub(plural_pattern," ",text)
            
            # tokenize
            tokenizer = RegexpTokenizer(r'\w+')
            processed_text = tokenizer.tokenize(text)

            # remove stop words
            processed_text = [text.lower() for text in processed_text if text.lower() 
                              not in stopwords.words('english')]
            
            # remove non-descript recipe words
            if nondescript == 1 and drop_words != None:
                processed_text = [text.lower() for text in processed_text if text.lower()
                                 not in drop_words]
                
            # append all each list that to describe an ingredient of the recipe
            ingredients.append(processed_text)

        # joined space-separated strings
        # attach all modifiers that describe each ingredient (non-separated)
        clean_ingredients = [" ".join(word) for word in ingredients]

        # append all ingredients for each recipe
        recipes_ingredients.append(clean_ingredients)    
    
    recipes_ingredients = [" ".join(ingredient) for ingredient in recipes_ingredients]
    recipes_ingredients = pd.DataFrame(recipes_ingredients)
    return recipes_ingredients

In [23]:
process_ingredients(short_recipes['ingredientLines'][:5])

,0
0,cup baby bella mushroom cup arborio rice tsp o...
1,lb pork country style rib fat cut x cubes oz c...
2,whole garlic heads tablespoon plu teaspoon ext...
3,garlic clove cooking spray potatoe cut inch sl...
4,pound lean ground beef tablespoon worcestershi...


In [64]:
# multi-threaded
def multi_process_ingredients(recipes,join=1,nondescript=0,drop_words=None):
    # create the patternsu

    # import punctuation characters
    # to remove all punctuation
    punct = string.punctuation
    punct_pattern = r"[{}]".format(punct)

    # to remove all numbers
    number_pattern = r"\d+\s"

    # embedded numbers
    embed_num_pattern = r".\d+."
    
    # removed prep methods
    prep_pattern = r"[a-z]+ed"
    
    # strip pluralization
    plural_pattern = r"s\s"
    
    # strip -ly
    ly_pattern = r"[a-z]+ly"
    
    recipes_ingredients = []
    ingredients = []

    for item in recipes:

        # strip punctuation
        text = re.sub(punct_pattern," ",item)
        # strip standalone numbers
        text = re.sub(number_pattern,"",text)
        # strip embedded numbers
        text = re.sub(embed_num_pattern,"",text)
        # strip preparation methods
        text = re.sub(prep_pattern,"",text)
        # strip pluralization
        text = re.sub(plural_pattern," ",text)
        # strip ly
        text = re.sub(ly_pattern,"",text)

        # tokenize
        tokenizer = RegexpTokenizer(r'\w+')
        processed_text = tokenizer.tokenize(text)

        # remove stop words
        processed_text = [text.lower() for text in processed_text if text.lower() 
                          not in stopwords.words('english')]

        # remove non-descript recipe words
        if nondescript == 1 and drop_words != None:
            processed_text = [text.lower() for text in processed_text if text.lower()
                             not in drop_words]

        # append all each list that to describe an ingredient of the recipe
        ingredients.append(processed_text)

    # joined space-separated strings
    # attach all modifiers that describe each ingredient (non-separated)
    clean_ingredients = [" ".join(word) for word in ingredients]

    # append all ingredients for each recipe
    recipes_ingredients.append(clean_ingredients)    
    
    if join == 0:
        pass
    else:
        recipes_ingredients = [" ".join(ingredient) for ingredient in recipes_ingredients]
    
    return recipes_ingredients

In [45]:
num_cores = multiprocessing.cpu_count()
inputs = short_recipes['ingredientLines']

if __name__ == "__main__":
    recipes = Parallel(n_jobs=num_cores)(delayed(multi_process_ingredients)(i) for i in inputs)

In [46]:
# 1 list of ingredients for each recipe
recipes = [" ".join(recipe) for recipe in recipes]
recipes = pd.DataFrame(recipes)

In [47]:
recipes.head()

,0
0,cup baby bella mushroom cup arborio rice tsp o...
1,lb pork country style rib fat cut x cubes oz c...
2,whole garlic heads tablespoon plu teaspoon ext...
3,garlic clove cooking spray potatoe cut inch sl...
4,pound lean ground beef tablespoon worcestershi...


In [48]:
# word counts
# get the words that occur most often in recipes
# these are candidates for removal in order to simplify the axis that we compare recipes

cvec = CountVectorizer()
cvecdata = cvec.fit_transform(recipes[0])

cvec_dense  = pd.DataFrame(cvecdata.todense(),
             columns=cvec.get_feature_names())

word_count = cvec_dense.sum(axis=0)    
cw = word_count.sort_values(ascending = False)
print(cw[0:100])

cw_dict = dict(cw)

cup           76809
teaspoon      46185
tablespoon    40129
pepper        26448
salt          26441
fresh         22550
ounce         18764
ground        18012
oil           17923
garlic        13560
onion         13473
pound         11820
cheese        11623
tsp           11288
olive         11173
large         10415
black         10162
sauce          9876
sugar          9869
clove          9588
cut            9493
chicken        8942
inch           8085
tbsp           7987
green          7694
powder         7561
butter         7550
juice          7408
taste          7385
oz             7179
              ...  
slice          2908
beef           2889
brown          2883
sour           2879
parmesan       2860
bell           2771
cooking        2770
paste          2727
frozen         2580
mustard        2573
italian        2563
vanilla        2548
chile          2542
garnish        2523
yellow         2512
oregano        2474
thick          2418
spray          2344
thyme          2331


In [49]:
len(cw)

9385

In [50]:
# quick function to manually evaluate words that ought to be removed
# https://stackoverflow.com/questions/5844672/delete-an-item-from-a-dictionary

def removekey(d, key):
    r = dict(d)
    del r[key]
    return r

def eval_words(word_list):
    keeps = []
    nondescript = []
    
    nondescript_words = [] 
    keep_words = []
    
    for key,value in word_list.items():
            word_eval = input('Keep {}: {}, y or n?'.format(key,value))
        
            if word_eval == 'n':
                nondescript_words.append(key)
            else:
                keep_words.append(key)
            
            remaining_list = removekey(word_list,key)
            
            if len(nondescript_words) % 100 == 0:
                nondescript = nondescript + nondescript_words
                keeps = keeps + keep_words
                
                # empty holding lists
                keep_words = []
                nondescript_words = []
                
                prompt_continue = input('Continue: yes or no?')
                if prompt_continue == "yes":
                    pass
                else:
                    return keeps, nondescript,remaining_list
    
    
    return keep_words, nondescript_words, remaining_list

In [51]:
keep, drop, remain = eval_words(cw)

Keep cup: 76809, y or n?n
Keep teaspoon: 46185, y or n?
Keep tablespoon: 40129, y or n?n
Keep pepper: 26448, y or n?
Keep salt: 26441, y or n?
Keep fresh: 22550, y or n?
Keep ounce: 18764, y or n?
Keep ground: 18012, y or n?
Keep oil: 17923, y or n?
Keep garlic: 13560, y or n?
Keep onion: 13473, y or n?
Keep pound: 11820, y or n?
Keep cheese: 11623, y or n?
Keep tsp: 11288, y or n?
Keep olive: 11173, y or n?
Keep large: 10415, y or n?
Keep black: 10162, y or n?
Keep sauce: 9876, y or n?
Keep sugar: 9869, y or n?
Keep clove: 9588, y or n?
Keep cut: 9493, y or n?
Keep chicken: 8942, y or n?
Keep inch: 8085, y or n?
Keep tbsp: 7987, y or n?
Keep green: 7694, y or n?
Keep powder: 7561, y or n?
Keep butter: 7550, y or n?
Keep juice: 7408, y or n?
Keep taste: 7385, y or n?n
Keep oz: 7179, y or n?
Keep white: 6825, y or n?
Keep cream: 6440, y or n?
Keep flour: 6259, y or n?
Keep water: 6145, y or n?n
Keep small: 5883, y or n?n
Keep ium: 5641, y or n?n
Keep cilantro: 5616, y or n?n
Keep lemon:

In [53]:
drop[:5]

['cup', 'tablespoon', 'taste', 'water', 'small']

In [ ]:
num_cores = multiprocessing.cpu_count()
inputs = short_recipes['ingredientLines']

if __name__ == "__main__":
    recipes = Parallel(n_jobs=num_cores)(delayed(multi_process_ingredients)(i,join=0,nondescript=1,drop_words=drop)
                                                 for i in inputs)

In [66]:
recipes[0]

[['baby bella mushroom',
  'arborio',
  'tsp olive oil',
  'tsp butter',
  'shallot',
  'white',
  'chicken',
  'salt pepper',
  'salt pepper',
  'cheese',
  'tbsp']]

In [67]:
recipes = ["|".join(recipe) for recipe in recipes]
recipes = pd.DataFrame(recipes)

TypeError: sequence item 0: expected str instance, list found

In [61]:
recipes

,0
0,baby bella mushroom arborio tsp olive oil tsp ...
1,country style rib cut x cubes oz sauce white j...
2,garlic heads teaspoon olive oil madeira chicke...
3,garlic clove potatoe cut inch pound butter te...
4,pound lean ground worcestershire sauce liquid ...
5,pound turkey maple syrup bourbon pickling spic...
6,tenderloin pound center cut silver skin worces...
7,sugar amaretto almond liqueur ounce dark cherr...
8,½ wheat crumb made ¾ wheat see step 1 pound ou...
9,wild chicken tenders green onions almonds tar...


I think I need to process ingredients 2 ways. As I did it above, it's fine. I lump all the ingredients together in order to run it through countvectorizer. 

When I prep data for the recommendation engine, I think I need to treat each ingredient, rather than each word as the unit of comparison. Still subject to analysis. I can probably stand it up as.

### More EDA

In [ ]:
# distribution of ingredients in recipes

In [ ]:
# most common, least common ingredients
# before and after non-descript word removal

# 

## Recommendation Engine

In [ ]:
# figure out input format?
# maybe input prompt
# or url scrape?

In [ ]:
# inventory the differences
# suggest additional ingredients in order to complete both recipes